# Importing required libraries

In [10]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import lxml.html as lh
import time
import urllib.request
import argparse
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
import coloredlogs
import sys
from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)

# Setting timezone and Chrome WebDriver
### Note: Works only on Chrome 81

In [11]:
tz = pytz.timezone('Asia/Kolkata')
path1 = os.getcwd()
path = path1 + '/chromedriver'
ignored_exceptions=(StaleElementReferenceException,)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(executable_path=path , options=options)

2020-05-22 12:25:35 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG POST http://127.0.0.1:51273/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"excludeSwitches": ["enable-logging"], "extensions": [], "args": ["--no-sandbox", "--disable-dev-shm-usage", "--headless"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"excludeSwitches": ["enable-logging"], "extensions": [], "args": ["--no-sandbox", "--disable-dev-shm-usage", "--headless"]}}}
2020-05-22 12:25:35 darkdevil urllib3.connectionpool[19390] DEBUG Starting new HTTP connection (1): 127.0.0.1:51273
2020-05-22 12:25:42 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "POST /session HTTP/1.1" 200 679
2020-05-22 12:25:42 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request


In [12]:
from trading_bot.agent import Agent
model_name = 'model_debug_50'
window_size = 10
debug = True
agent = Agent(window_size, pretrained=True, model_name=model_name)

# Function that returns Price of the given URL

In [13]:
def Real(url,count):
    if count == 0:
        driver.get(url)
    else:
        driver.refresh()
        time.sleep(15)

    infile = driver.page_source
    doc = lh.fromstring(infile)
    live = doc.xpath('/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[3]/div/div/span[1]')
    live = float(live[0].text.replace(',',''))
    return live 

# Function that plots the graph

In [14]:
def visualize(df, history, title="trading session"):
    # Add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 2, max(max(df['actual']), max(df['position'])) + 2), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='hoursminutesseconds(time):T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x = alt.X('hoursminutesseconds(time):O', title='Time'),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [15]:
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
    action = agent.act(state, is_eval=True)
    # BUY
    if action == 1:
        agent.inventory.append(data[t])
        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))
    # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta
        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
    # HOLD
    else:
        history.append((data[t], "HOLD"))

    agent.memory.append((state, action, reward, next_state))
    return total_profit

# The main function

In [49]:
coloredlogs.install(level="DEBUG")
switch_k_backend_device()
args = argparse.Namespace(ticker='RELIANCE')
try:
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    ticker = args.ticker + '.NS'
    price = []
    window_size = 2
    time_array = []
    time_now = datetime.datetime.now(tz).time()
    d = {}
    while(datetime.time(9, 14, tzinfo=tz) < time_now < datetime.time(15, 31, tzinfo=tz)):
        url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(ticker,ticker)
        print(count)
        live = Real(url,count)
        count+=1        
        x = datetime.datetime.now(tz).isoformat().split('.')[0]
        price.append(live)
        time_array.append(x)
        if count < window_size:
           continue
        model_name='model_debug_50'
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
        state = next_state
        
        if count > 12:
            break
    #price = price[:len(time_array)]
    d = {'time': time_array, 'actual': price}  
    df = pd.DataFrame(d) 
    df.to_csv('new.csv', columns=["time", "actual"])
except KeyboardInterrupt:
    print("Aborted")    
visualize(df, history, title="trading session")

2020-05-22 12:55:10 darkdevil root[19390] DEBUG switching to TensorFlow for CPU
2020-05-22 12:55:10 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG POST http://127.0.0.1:51273/session/d481ffcf4b322db9c104351bac4d399f/url {"url": "https://finance.yahoo.com/quote/RELIANCE.NS?p=RELIANCE.NS&.tsrc=fin-srch"}


0


2020-05-22 12:55:11 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "POST /session/d481ffcf4b322db9c104351bac4d399f/url HTTP/1.1" 200 14
2020-05-22 12:55:11 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request
2020-05-22 12:55:11 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG GET http://127.0.0.1:51273/session/d481ffcf4b322db9c104351bac4d399f/source {}
2020-05-22 12:55:11 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "GET /session/d481ffcf4b322db9c104351bac4d399f/source HTTP/1.1" 200 848307
2020-05-22 12:55:11 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request
2020-05-22 12:55:11 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG POST http://127.0.0.1:51273/session/d481ffcf4b322db9c104351bac4d399f/refresh {}


1


2020-05-22 12:55:13 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "POST /session/d481ffcf4b322db9c104351bac4d399f/refresh HTTP/1.1" 200 14
2020-05-22 12:55:13 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request
2020-05-22 12:55:28 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG GET http://127.0.0.1:51273/session/d481ffcf4b322db9c104351bac4d399f/source {}
2020-05-22 12:55:28 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "GET /session/d481ffcf4b322db9c104351bac4d399f/source HTTP/1.1" 200 887629
2020-05-22 12:55:28 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request
2020-05-22 12:55:29 darkdevil root[19390] INFO model_debug_50: USELESS

2020-05-22 12:55:29 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG POST http://127.0.0.1:51273/session/d481ffcf4b322db9c104351bac4d399f/refresh {}


...

11


2020-05-22 12:58:07 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "POST /session/d481ffcf4b322db9c104351bac4d399f/refresh HTTP/1.1" 200 14
2020-05-22 12:58:07 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request
2020-05-22 12:58:22 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG GET http://127.0.0.1:51273/session/d481ffcf4b322db9c104351bac4d399f/source {}
2020-05-22 12:58:22 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "GET /session/d481ffcf4b322db9c104351bac4d399f/source HTTP/1.1" 200 887685
2020-05-22 12:58:22 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request
2020-05-22 12:58:23 darkdevil root[19390] INFO model_debug_50: USELESS

2020-05-22 12:58:23 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG POST http://127.0.0.1:51273/session/d481ffcf4b322db9c104351bac4d399f/refresh {}


12


2020-05-22 12:58:24 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "POST /session/d481ffcf4b322db9c104351bac4d399f/refresh HTTP/1.1" 200 14
2020-05-22 12:58:24 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request
2020-05-22 12:58:39 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG GET http://127.0.0.1:51273/session/d481ffcf4b322db9c104351bac4d399f/source {}
2020-05-22 12:58:39 darkdevil urllib3.connectionpool[19390] DEBUG http://127.0.0.1:51273 "GET /session/d481ffcf4b322db9c104351bac4d399f/source HTTP/1.1" 200 886655
2020-05-22 12:58:39 darkdevil selenium.webdriver.remote.remote_connection[19390] DEBUG Finished Request
2020-05-22 12:58:40 darkdevil root[19390] INFO model_debug_50: USELESS



alt.LayerChart(...)

In [43]:
time_array

['2020-05-22T12:49:23',
 '2020-05-22T12:49:40',
 '2020-05-22T12:49:57',
 '2020-05-22T12:50:15']

In [44]:
history

[(1433.5, 'BUY'), (1434.45, 'BUY'), (1433.35, 'BUY'), (1433.6, 'BUY')]

In [45]:
price

[1433.5, 1434.45, 1433.35, 1433.6]

In [46]:
df

,time,actual
0,2020-05-22T12:49:23,1433.50
1,2020-05-22T12:49:40,1434.45
2,2020-05-22T12:49:57,1433.35
3,2020-05-22T12:50:15,1433.60


In [47]:
visualize(df, history, title="trading session")

ValueError: Length of values does not match length of index